In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool

In [2]:
from helpers import CVFConfigDataset

from torch.utils.data import DataLoader, random_split

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""

    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        # H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes).to(device), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(x, A + torch.eye(num_nodes).to(device)))
        return x

In [5]:
class GCNByHand(nn.Module):
    def __init__(self, N, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.out = torch.nn.Linear(N, out_channels, bias=True)

    def forward(self, x, A):
        x = self.conv1(x, A)
        x = torch.relu(x)
        x = self.conv2(x, A)
        x = torch.relu(x)
        x = self.out(x)
        x = torch.relu(x)
        # x = torch.sigmoid(x) * 9
        # print("x after output layer\n", x)
        # x = global_mean_pool(x, torch.zeros(x.size(1)).long())  # all are from graph 0, single graph
        return x
    
    def fit(self, train_loader, epochs):
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01) # weight_decay is a L2 regularization parameter
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in train_loader:
                x = batch[0].to(device)
                y = batch[1].to(device)
                optimizer.zero_grad()
                out = self(x, train_loader.dataset.dataset.A.to(device))
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Training set | Epoch:", epoch, "Loss:", total_loss / count)
            
            # self.eval()
            # with torch.no_grad():
            #     total_loss = 0
            #     count = 0
            #     for batch in validation_loader:
            #         x = batch[0].to(device)
            #         x = add_graph_properties(x)
            #         y = batch[1].to(device)
            #         y = y.unsqueeze(0).reshape(-1, 1, 1).float()
            #         optimizer.zero_grad()
            #         out = self(x)
            #         loss = criterion(out, y)
            #         total_loss += loss
            #         count += 1

            # print("Validatn set | Epoch:", epoch, "Loss:", total_loss / count)
            # print()

In [6]:
num_nodes = 10       # N
num_features = 1     # D
num_labels = 1       # O
batch_size = 10      # B

# the following needs to be replaced by some node embeddings
# x = torch.randn((batch_size, num_features, num_nodes)) # B x D x N

# A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(N=num_nodes, in_channels=num_features, out_channels=num_labels)
model.to(device)

# out = model(x, A)   # B x D x O

# print("output", out.shape)

GCNByHand(
  (conv1): GCNConvByHand(
    (linear): Linear(in_features=1, out_features=1, bias=True)
  )
  (conv2): GCNConvByHand(
    (linear): Linear(in_features=1, out_features=1, bias=True)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)

A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
# preactivation

out_wt = np.array(model.out.weight.detach().numpy())
out_bias = np.array(model.out.bias.detach().numpy())
preactivation @ out_wt.transpose() + out_bias

In [7]:
# dataset = CVFConfigDataset(
#     "dijkstra",
#     "implicit_graph_n10_config_rank_dataset.csv",
#     "implicit_graph_n10_A.json",
#     3,
#     one_hot_encode=False,
# )
dataset = CVFConfigDataset(
    "coloring",
    "graph_1_config_rank_dataset.csv",
    "graph_1_A.json",
    11,
    one_hot_encode=False,
)

In [8]:
len(dataset)

5120

In [9]:
train_split = 0.95
test_split = 0.05

train_set, test_set = random_split(
    dataset, [train_split, test_split]
)

train_loader = DataLoader(train_set, batch_size=50, shuffle=True)
test_loader = DataLoader(test_set, batch_size=10, shuffle=False)

In [10]:
train_loader.dataset.dataset.A

tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
model.fit(train_loader, 20)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

In [ ]:
# testing
torch.no_grad()
# torch.set_printoptions(profile="full")

total_matched = 0

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    predicted = model(x, test_loader.dataset.dataset.A.to(device))
    print(y[0], predicted[0])
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched

print("Total matched", total_matched, "out of", len(test_set), "| Accuracy", round(total_matched/len(test_set) * 100, 4), "%")

tensor([[5.]], device='cuda:0') tensor([[5.1573]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.7297]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[1.1777]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([[0.]], device='cuda:0') tensor([[0.]], device='cuda:0', grad_fn=<Selec